In [1]:
import pickle 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import KFold, cross_val_score


## 뉴스 기사를 이용한 성별 선호도 분류 

#### Data Loading 

- DataFrame has four columns, "Date", "Title", "sex", "Body" 

In [2]:
with open("./article_201904.bin", 'rb') as f :
    df = pickle.load(f)
df.tail()

df = df[["Body", "age_sex"]]

In [3]:
df.tail()

,Body,age_sex
3995,윤석열 검찰총장 후보자가 8일 국회에서 열린 국회 인사청문회에서 선서를 하고 있다....,50&male
3996,서울 용산구 이촌파출소 [연합뉴스 자료 사진]\n \n\n(서울=연...,50&male
3997,【서울=뉴시스】 박영태 기자 = 8일 오전 서울 여의도 국회에서 열린 검찰총장후보자...,50&male
3998,"""트럼프가 놔두겠나""…日 '횡포' 일주일, 초조감 속 낙관론도 (CG) [연합뉴스T...",50&male
3999,대만의 반도체 파운드리(위탁생산) 업체 TSMC 공장 전경(TSMC 제공) © Ne...,50&male


#### Dataset split 

- 80% -> Training 
- 20% -> Test

In [4]:
def dataset_split(df):
    """
    Split df to train/test 
    
    Argument :
    df = pandas.DataFrame
    return : 
    Trainset, Testset
    """
    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)
    train_idx = idx[:int(df.shape[0]*0.8)]
    test_idx = idx[int(df.shape[0]*0.8):]
    
    train_df = df.loc[train_idx]
    test_df = df.loc[test_idx]
    
    return train_df, test_df 

train_df, test_df = dataset_split(df)

In [5]:
test_df.tail()

,Body,age_sex
3385,<앵커>\n가수 승리가 투자자들에게 성접대를 했다는 의혹이 갈수록 커지고 있습니다....,40&male
292,지금 현재 보이는 우측의 화면이 불이 났을 때의 모습이고요.\n좌측에 있는 화면은 ...,30&female
5764,도로 위 오렌지 정리하는 경찰 [부산경찰청 제공]\n \n\n(부산...,20&female
1851,"[오마이뉴스 글:한승희, 편집:김혜리]\n이 자리의 주인은 누구인가? 약자를 배려하...",30&female
3255,배우 문채원(33)이 성관계 동영상 불법 촬영과 유포 혐의를 받는 가수 정준영(30...,30&male


#### Try Simple Counting and vectorizing 
> scikit learn.countvertorizer 

- 가장 기본적인 것, 문제가 단순하다고 판단되어, max_df, min_df는 설정하지 않았음
- tokenizer도 기본으로 사용 
---------
###### Countervector화 시킨 후, Multinomial NB모델 

* 타이틀을 input 하면, 어떤 성별이 좋아 할 지 예측 한다. 

:: Accuracy = 0.9292 | 정확도만 고려 해도 괜찮은 문제라고 생각한다. 


- 각 성별별로 어떤 단어들이 많이 있을까 ? -> wordCloud 
- 성별을 분류하는데 기준이 되는 단어 ? -> 어떻게 알 수 있을까 ? --> Theta값의 차이가 큰 것  -> 비율로 나타내어 확인 해보자, 


##### 합성 명사 추출 


In [6]:
from ckonlpy.tag import Twitter
from collections import defaultdict
# import sys

twitter = Twitter(use_twitter_dictionary=True)

tagged_data = []

for data in train_df.Body:
    tagged_data.append(twitter.pos(data))

/Users/MAC/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [7]:
josa = list(twitter.dictionary._pos2words['Josa'])

bigram_compound = set()
trigram_compound = set()

for pos, raw in zip(tagged_data, train_df.Body):
    for i in range(len(pos)-2):
        if pos[i][1] == 'Noun' and pos[i+1][1] == 'Noun':
            string = pos[i][0] + pos[i+1][0]
            if len(re.findall(string, raw)) >= 3 and (pos[i+1][0] not in josa): 
                bigram_compound.add(string)

for pos, raw in zip(tagged_data, train_df):
    for i in range(len(pos)-3):
        if pos[i][1] == 'Noun' and pos[i+1][1] == 'Noun' and pos[i+2][1] == 'Noun':
            string = pos[i][0] + pos[i+1][0] + pos[i+2][0]
            if len(re.findall(string, raw)) >= 3 and (pos[i+2][0] not in josa):
                trigram_compound.add(string)

compound = bigram_compound | trigram_compound

#### Tokenizer 생성 

In [8]:
twitter.add_dictionary(compound, 'Noun')

In [9]:
import stopwords

stop_words = stopwords.get_stopwords(language='ko')

def mytokenizer(raw_sentence):
    tagged_sentences = twitter.pos(raw_sentence)
    tokenized_sentence = []
    for word in tagged_sentences:
        if (word[1] not in ["Eomi", "Josa", "PreEomi"]) and (word[0] not in stop_words) and (len(word[0]) > 1):
            tokenized_sentence.append(word[0])           
    return tokenized_sentence

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#corpus 
corpus = [cor for cor in train_df.Body.values]

# countvectorizer
c_bow = CountVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, lowercase=False)
c_bow_bigram = CountVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, ngram_range=(1, 2), lowercase=False)
c_bow_trigram = CountVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, ngram_range=(1, 3), lowercase=False)

# tfidfvectorizer
t_bow = TfidfVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, lowercase=False)
t_bow_bigram = TfidfVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, ngram_range=(1, 2), lowercase=False)
t_bow_trigram = TfidfVectorizer(tokenizer=mytokenizer, min_df=30, max_df=0.7, ngram_range=(1, 3), lowercase=False)

# fitting
c_bow_feature = c_bow.fit_transform(corpus).todense()
c_bow_bigram_feature = c_bow_bigram.fit_transform(corpus).todense()
c_bow_trigram_feature = c_bow_trigram.fit_transform(corpus).todense()

t_bow_feature = t_bow.fit_transform(corpus).todense()
t_bow_bigram_feature = t_bow_bigram.fit_transform(corpus).todense()
t_bow_trigram_feature = t_bow_trigram.fit_transform(corpus).todense()

del corpus

In [11]:
c_bow_feature.shape, t_bow_feature.shape

((4928, 6251), (4928, 6251))

In [12]:
c_bow_bigram_feature.shape, t_bow_bigram_feature.shape

((4928, 7685), (4928, 7685))

In [13]:
c_bow_trigram_feature.shape, t_bow_trigram_feature.shape

((4928, 7807), (4928, 7807))

In [14]:
df.age_sex.unique()

array(['20&female', '20&male', '30&female', '30&male', '40&female',
       '40&male', '50&female', '50&male'], dtype=object)

In [15]:
# label_dict = {ax : i for i, ax in enumerate(df.age_sex.unique())}
# label_dict_decode = {i : ax for i, ax in enumerate(df.age_sex.unique())}

In [16]:
%%time
cv = KFold(5, shuffle=True)

# model = MultinomialNB()

features = [c_bow_feature, c_bow_bigram_feature, c_bow_trigram_feature, 
            t_bow_feature, t_bow_bigram_feature, t_bow_trigram_feature]
variations = ["Uni_countvectorizer", "Bi_countvectorizer", "Tri_countvectorizer",
             "Uni_TfIdfvectorizer", "Bi_TfIdfvectorizer", "Tri_TfIdfvectorizer"]
y = [0 if 'male' == ax.split("&")[1] else 1 for ax in train_df.age_sex.values]

for i, feature in enumerate(features):
    print("{} starts".format(variations[i]))
    model= MultinomialNB().fit(feature, y)
    pred = model.predict(feature)
    scores = cross_val_score(model, feature, y, cv=cv, scoring='accuracy')
    print("Model{} : mean score: {}\n".format(variations[i], np.mean(scores)))


Uni_countvectorizer starts
ModelUni_countvectorizer : mean score: 0.7207777926504051

Bi_countvectorizer starts
ModelBi_countvectorizer : mean score: 0.7228111325048135

Tri_countvectorizer starts
ModelTri_countvectorizer : mean score: 0.7248387063559889

Uni_TfIdfvectorizer starts
ModelUni_TfIdfvectorizer : mean score: 0.7221910812285705

Bi_TfIdfvectorizer starts
ModelBi_TfIdfvectorizer : mean score: 0.7266615870923899

Tri_TfIdfvectorizer starts
ModelTri_TfIdfvectorizer : mean score: 0.7266628226645112

CPU times: user 32.2 s, sys: 7.92 s, total: 40.1 s
Wall time: 14.9 s


#### 성적이 그나마 좋은 Tfidf_unigram 으로 vectorize한 feature를 사용한 모델 

In [17]:
y = [0 if 'male' == ax.split("&")[1] else 1 for ax in train_df.age_sex.values]

In [18]:
model = MultinomialNB().fit(t_bow_feature, y)
model

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
tokens = t_bow.get_feature_names()

male_token_count = model.feature_count_[0] + model.alpha
female_token_count = model.feature_count_[1] + model.alpha


token_df = pd.DataFrame({"tokens" : tokens,
                          "male" : male_token_count,
                          "female" : female_token_count}).set_index("tokens")

token_df.sample(10) ## Tfidf방식으로 vectorize했기 때문에 실수값으로 나온다 .

,male,female
tokens,,
받았,3.129637,3.302372
몰고,3.031106,3.022037
부채,3.148583,1.609324
구심,1.663308,1.469871
2002년,2.522989,2.294198
찍는,2.673769,2.699654
바깥,2.206476,3.196799
훔쳐,2.461853,2.626944
일로,4.316352,3.250145


#### 남녀간의 비교 

In [20]:
male_total_count = sum(token_df.male)
female_total_count = sum(token_df.female)

token_df["male_probablility"] = token_df.male / male_total_count
token_df["female_probablility"] = token_df.female / female_total_count
token_df["male_prior"] = token_df.male_probablility / token_df.female_probablility
token_df["female_prior"] = token_df.female_probablility / token_df.male_probablility
token_df.sample(15)

,male,female,male_probablility,female_probablility,male_prior,female_prior
tokens,,,,,,
적고,1.453465,1.935805,0.000051,0.000068,0.744330,1.343490
포즈,1.954484,2.270155,0.000068,0.000080,0.853492,1.171658
올라온,3.047534,3.460929,0.000107,0.000122,0.872928,1.145570
문제점,2.709076,2.632140,0.000095,0.000093,1.020316,0.980088
1조,4.817738,2.070330,0.000169,0.000073,2.306886,0.433485
학과,3.014446,4.258956,0.000106,0.000150,0.701660,1.425191
포인트,8.184544,5.222248,0.000287,0.000184,1.553673,0.643636
않겠다는,2.656224,1.804200,0.000093,0.000064,1.459495,0.685168
신용카드,4.852807,3.286090,0.000170,0.000116,1.463983,0.683068


In [21]:
token_df[["male_prior", "female_prior"]].sample(10)

,male_prior,female_prior
tokens,,
객실,0.675453,1.480489
밝힐,0.786583,1.271321
끼리,0.821323,1.217548
국토교통부,1.222320,0.818116
좋습니다,0.558969,1.789007
?',0.951650,1.050807
마구,1.065850,0.938218
임무,3.195786,0.312912
있던,0.781350,1.279836


#### 성별 워드클라우드 생성 

In [22]:
from wordcloud import WordCloud

male_dict = {token: ratio for token, ratio in zip(token_df.index, token_df.male_prior)}
female_dict = {token: ratio for token, ratio in zip(token_df.index, token_df.female_prior)}

male_wordcloud = WordCloud(font_path="/Library/Fonts/AppleGothic.ttf",
                   background_color="white",
                   width=800,
                   height=400).generate_from_frequencies(male_dict)

female_wordcloud = WordCloud(font_path="/Library/Fonts/AppleGothic.ttf",
                   background_color="white",
                   width=800,
                   height=400).generate_from_frequencies(female_dict)

In [23]:
plt.figure(figsize=(17, 9), dpi=700)
plt.imshow(male_wordcloud)
plt.axis("off")

plt.savefig("male_prefered_words.png", transparent=True, bbox_inches="tight")

In [24]:
plt.figure(figsize=(17, 9), dpi=700)
plt.imshow(female_wordcloud)
plt.axis("off")

plt.savefig("female_prefered_words.png", transparent=True, bbox_inches="tight")